In [ ]:
# Formatted with Black
!pip install black[jupyter]
!black /content/drive/MyDrive/Capstone/tdcn-test.ipynb

# TDCN 이미지 예측 모델 구현

In [2]:
import os
import warnings
from google.colab import drive

import pandas as pd
from tqdm.auto import tqdm, trange
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [ ]:
drive.mount("/content/drive")
warnings.simplefilter("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8
EPOCHS = 110
CWD = "/content/drive/MyDrive/Capstone"


def join_path(*args):
    return os.path.join(CWD, *args)


TRAIN_CSV = [
    join_path("data", csv)
    for csv in ["feat_xtrain.csv", "pose_xtrain.csv", "ytrain.csv"]
]
TEST_CSV = [
    join_path("data", csv) for csv in ["feat_xtest.csv", "pose_xtest.csv", "ytest.csv"]
]

# Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, x1_file, x2_file, y_file, mode=None):
        x1_df = pd.read_csv(x1_file)  # landmark data
        x2_df = pd.read_csv(x2_file)  # pose data
        y_df = pd.read_csv(y_file)

        x1 = x1_df.values
        x2 = x2_df.values
        if mode == "train":
            y = y_df.iloc[:, 1].values
        else:
            y = y_df.values

        self.x1_data = torch.FloatTensor(x1)
        self.x2_data = torch.FloatTensor(x2)
        self.y_data = torch.IntTensor(y)

    def __len__(self):
        return len(self.y_data)

    def __getitem__(self, index):
        return (
            self.x1_data[index * 5000 : (index + 1) * 5000],
            self.x2_data[index * 5000 : (index + 1) * 5000],
        ), self.y_data[index]

In [5]:
# 데이터셋
training_data = CustomDataset(*TRAIN_CSV, mode="train")
test_data = CustomDataset(*TEST_CSV)

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

# Dilated Conv Block

In [6]:
class DilatedConvBlock(nn.Module):
    def __init__(self, has_BN=False, batch_size=8):
        super(DilatedConvBlock, self).__init__()
        self._c = 1
        self._has_BN = has_BN

        self.dilated_conv1 = self.dilated_conv(1)
        self.dilated_conv2 = self.dilated_conv(2)
        self.dilated_conv3 = self.dilated_conv(4)
        self.conv1d = nn.Conv2d(self._c, self._c, kernel_size=(1, 1))
        self.BN = nn.BatchNorm2d(self._c, affine=True)

    def dilated_conv(self, d):
        return nn.Conv2d(
            self._c,
            self._c,
            kernel_size=(3, 3),
            stride=1,
            padding="same",
            dilation=d,
            bias=True,
            padding_mode="zeros",
        )

    def forward(self, x):
        x_1d = self.conv1d(x)

        # DCN with d=1
        x_2d_1 = self.dilated_conv1(x)
        x_2d_2 = self.dilated_conv1(x)
        x_2d = x_2d_1 + x_2d_2
        x_2d = F.elu(x_2d)

        # DCN with d=2
        x_2d_1 = self.dilated_conv2(x_2d)
        x_2d_2 = self.dilated_conv2(x_2d)
        x_2d = x_2d_1 + x_2d_2
        x_2d = F.elu(x_2d)

        # DCN with d=4
        x_2d_1 = self.dilated_conv3(x_2d)
        x_2d_2 = self.dilated_conv3(x_2d)
        x_2d = x_2d_1 + x_2d_2
        x_2d = F.elu(x_2d)

        x = x_2d_1 + x_2d_2
        
        if self._has_BN:
            x = self.BN(x)
        return x

# TDCN + FWA + Prediction

In [7]:
class PredictionModel(nn.Module):
    def __init__(self, batch_size=8):
        super(PredictionModel, self).__init__()

        self.TDCN = nn.Sequential(
            DilatedConvBlock(has_BN=True, batch_size=BATCH_SIZE),
            nn.MaxPool2d(kernel_size=(2, 1), stride=None),
            DilatedConvBlock(has_BN=True, batch_size=BATCH_SIZE),
            nn.MaxPool2d(kernel_size=(2, 1), stride=None),
            DilatedConvBlock(has_BN=True, batch_size=BATCH_SIZE),
            nn.MaxPool2d(kernel_size=(2, 1), stride=None),
            DilatedConvBlock(has_BN=True, batch_size=BATCH_SIZE),
            nn.MaxPool2d(kernel_size=(2, 1), stride=None),
            DilatedConvBlock(has_BN=False),
        )
        self.classify = nn.Sequential(
            nn.Linear(142, 64), nn.Linear(64, 32), nn.Linear(32, 2), nn.Softmax(dim=3)
        )

        self.attention_layer = nn.Sequential(
            nn.Linear(142, 142),
            nn.ReLU(inplace=True),
            nn.Linear(142, 142),
            nn.Sigmoid(),
        )

    def global_average_pooling(self, x):
        return torch.mean(x, dim=2)

    def FWA(self, x1, x2):
        x = torch.concat((x1, x2), dim=3)

        x_ = self.global_average_pooling(x)
        x_ = self.attention_layer(x_)
        x_ = torch.unsqueeze(x_, 2)
        x = torch.mul(x, x_)

        return x

    def forward(self, x_landmark, x_pose):
        x_landmark = self.TDCN(x_landmark)
        x_pose = self.TDCN(x_pose)
        x = self.FWA(x_landmark, x_pose)
        score = self.classify(x)
        score = score[:, :, :, 0].mean(dim=2)
        return score

# Train

In [8]:
loss_fn = nn.BCELoss()
model = PredictionModel().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=2e-5, momentum=0.9)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        x1, x2 = X
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        y = y.to(torch.float32)
        pred = model(
            x1.unsqueeze(0).permute(1, 0, 2, 3), x2.unsqueeze(0).permute(1, 0, 2, 3)
        )
        loss = loss_fn(pred, y.unsqueeze(1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = loss.item()
    return loss


def test(dataloader, model, loss_fn, return_metrics=False):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0
    real_values = torch.IntTensor().to(device)
    pred_values = torch.IntTensor().to(device)

    with torch.no_grad():
        for X, y in dataloader:
            x1, x2 = X
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)
            y = torch.Tensor(y)
            pred = model(
                x1.unsqueeze(0).permute(1, 0, 2, 3), x2.unsqueeze(0).permute(1, 0, 2, 3)
            )

            test_loss += loss_fn(pred, y.to(torch.float32)).item()\

            if return_metrics:
                real_values = torch.cat((real_values, y), dim=0)
                pred_values = torch.cat(
                    (pred_values, torch.round(pred).to(torch.int32)), dim=0
                )

    if not return_metrics:
        # 학습 과정에서는 Loss 값만 확인합니다.
        test_loss /= num_batches
        return test_loss

    else:
        # 학습이 종료되고 성능 평가 지표를 확인합니다. 
        real_values = real_values.squeeze().cpu().detach().numpy()
        pred_values = pred_values.squeeze().cpu().detach().numpy()

        accuracy = accuracy_score(real_values, pred_values)
        f1 = f1_score(real_values, pred_values)
        f1_macro = f1_score(real_values, pred_values, average="macro")
        recall = recall_score(real_values, pred_values)
        precision = precision_score(real_values, pred_values)

        return {
            "accuracy": accuracy,
            "f1": f1,
            "f1-macro": f1_macro,
            "recall": recall,
            "precision": precision,
            "loss": test_loss
        }
    

In [12]:
epoch_progress = trange(EPOCHS)
tqdm.write("\nEpoch | Train Loss | Test Loss")
tqdm.write("-" * 30)

for epoch in epoch_progress:
    train_loss = train(train_dataloader, model, loss_fn, optimizer)
    test_loss = test(test_dataloader, model, loss_fn)

    if epoch % 20 == 0:
        tqdm.write(
            f"{epoch:5} | {train_loss:10.5f} | {test_loss:9.5f}"
        )

  0%|          | 0/110 [00:00<?, ?it/s]


Epoch | Train Loss | Test Loss
------------------------------
    0 |    0.59453 |   0.65510
   20 |    0.41144 |   0.60844
   40 |    0.39051 |   0.60603
   60 |    0.38992 |   0.60598
   80 |    0.39138 |   0.60614
  100 |    0.39015 |   0.60602


In [13]:
torch.save(model.state_dict(), join_path("tdcn.pth"))
torch.cuda.empty_cache()

# Evaluation

In [14]:
model = PredictionModel(BATCH_SIZE)
model.load_state_dict(torch.load(join_path("tdcn.pth")))
model.to(device)

PredictionModel(
  (TDCN): Sequential(
    (0): DilatedConvBlock(
      (dilated_conv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (dilated_conv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, dilation=(2, 2))
      (dilated_conv3): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, dilation=(4, 4))
      (conv1d): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
      (BN): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (2): DilatedConvBlock(
      (dilated_conv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (dilated_conv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, dilation=(2, 2))
      (dilated_conv3): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, dilation=(4, 4))
      (conv1d): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
      (BN)

In [15]:
metrics = test(test_dataloader, model, loss_fn, return_metrics=True)

print(f"Accuracy:  {metrics['accuracy']:.3f}")
print(f"F1-score:  {metrics['f1']:.3f}")
print(f"F1-macro:  {metrics['f1-macro']:.3f}")
print(f"Recall:    {metrics['recall']:.3f}")
print(f"Precision: {metrics['precision']:.3f}")

Accuracy:  0.702
F1-score:  0.000
F1-macro:  0.412
Recall:    0.000
Precision: 0.000
